In [1]:
import os, sys, torch
from transformers import AutoModel, AutoTokenizer
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate

/Users/yonatan/.conda/envs/nlp_final_project/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append("../")
from src.model_head import HistoricalTextDatingModel, create_model_head_config
from src.utils import init_tracker, DataLoadAndFilter


In [3]:
# 1) Set your checkpoint path here
ckpt_base_path = "../outputs/google-bert/bert-base-multilingual-cased/2025-09-15/23-08-03/"
ckpt_path = ckpt_base_path + "bert_DatePredictionHead_trainingFalse_encoder_hidden_size768_num_classes36_pooling_strategycls_min_date1000_max_date2024_normalize_outputTrue_dropout_rate0.1_frozen.pt"
config_path = ckpt_base_path + ".hydra/config.yaml"
if not os.path.exists(ckpt_path):
    raise FileNotFoundError(f"Checkpoint path does not exist: {ckpt_path}")
if not os.path.exists(config_path):
    raise FileNotFoundError(f"Config path does not exist: {config_path}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
ckpt = torch.load(ckpt_path, map_location=device)

In [5]:
cfg = OmegaConf.load(config_path)

In [15]:

# Load the model and tokenizer
tokenizer = instantiate(cfg.model.tokenizer)
encoder = instantiate(cfg.model.encoder)

model_head_config = create_model_head_config(**cfg.model.model_head.head_config)

# Load datasets using the data loader
data_loader = DataLoadAndFilter(cfg)
train_dataset, eval_dataset = data_loader.create_tokenized_datasets(tokenizer, base_path="../")

model_head_config["num_classes"] = len(train_dataset[0][1])


Error loading schema ../data/raw/SefariaData/Sefaria-Export-master/schemas/Sheet.json: Expecting value: line 1 column 1 (char 0)


Loading Sefaria texts: 100%|██████████| 100/100 [00:02<00:00, 37.49it/s]


Loaded 50 text samples from Sefaria dataset


Loading Ben Yehuda texts: 100%|██████████| 100/100 [00:00<00:00, 2762.83it/s]


In [16]:
model_head_config

{'hidden_sizes': [384, 128], 'dropout_rate': 0.1, 'activation': 'relu', 'pooling_strategy': 'cls', 'min_date': 1000, 'max_date': 2024, 'normalize_output': True, 'num_classes': 36}

In [17]:
model = HistoricalTextDatingModel(
    encoder=encoder,
    head_config=model_head_config,
    freeze_encoder=True,  # Freeze encoder to not change bert
)

In [18]:
def _clean_state_dict(state_dict: dict) -> dict:
    if not any(k.startswith("module.") for k in state_dict.keys()):
        return state_dict
    return {k.replace("module.", "", 1): v for k, v in state_dict.items()}

In [21]:
clean_ckpt = _clean_state_dict(ckpt)

In [22]:
missing, unexpected = model.load_state_dict(clean_ckpt, strict=False)
if missing:
    print(f"[load_state_dict] Missing keys: {len(missing)} (showing first 10)\n", missing[:10])
if unexpected:
    print(f"[load_state_dict] Unexpected keys: {len(unexpected)} (showing first 10)\n", unexpected[:10])
model.eval()


HistoricalTextDatingModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [23]:
print("Model loaded to:", device)


Model loaded to: cpu


In [30]:

def predict_date(text: str) -> float:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        preds, _ = model(input_ids=inputs["input_ids"], attention_mask=inputs.get("attention_mask"))
    # preds shape: (batch,) -> take scalar
    return preds[0]

In [ ]:
example_text = "This is a sample historical text snippet to test the model."
print("Predicted date:", predict_date(example_text))
